In [ ]:
import time
from joblib import Parallel, delayed

import pandas as pd
import numpy as np
import category_encoders as ce

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import RandomizedSearchCV, GroupKFold
from sklearn.utils import shuffle
from sklearn import preprocessing, pipeline

from sklearn.metrics import accuracy_score, precision_score, recall_score, balanced_accuracy_score, f1_score

In [ ]:
#Original dataset
data = pd.read_json(r'original-dataset.json', orient='split')

# Original dataset without tenders with partial collusion
data = pd.read_json(r'original-dataset-without-partials.json', orient='split')

cat_features = ['Site', 'Brazilian State']

In [ ]:
# #Enriched dataset
# data = pd.read_json(r'enriched-dataset.json', orient='split')

# # Enriched dataset without tenders with partial collusion
# data = pd.read_json(r'enriched-dataset-without-partials.json', orient='split')

# cat_features = ['Site', 'Brazilian State', 'natureza_juridica']

In [ ]:
data['Difference Bid/PTE'] = data['Bid_value'] - data['Pre-Tender Estimate (PTE)']

In [ ]:
encoders = [
    ce.BinaryEncoder(cols = cat_features, handle_unknown = 'value', return_df = True),
    ce.QuantileEncoder(cols = cat_features, handle_unknown = 'value', return_df = True),
    ce.OneHotEncoder(cols = cat_features, handle_unknown = 'value', return_df = True),
    ce.RankHotEncoder(cols = cat_features, handle_unknown = 'value', return_df = True),
    ce.HashingEncoder(cols = cat_features, return_df = True),
    ce.CountEncoder(cols = cat_features, handle_unknown = 'value', return_df = True)
]

In [ ]:
scalers = [
    preprocessing.StandardScaler(),
    preprocessing.MinMaxScaler(),
    preprocessing.MaxAbsScaler(),
    preprocessing.RobustScaler(quantile_range=(25,75)),
    preprocessing.PowerTransformer(method='yeo-johnson'),
    preprocessing.QuantileTransformer(output_distribution='uniform'),
    preprocessing.QuantileTransformer(output_distribution='normal'),
]

In [ ]:
models = {
    ExtraTreesClassifier(): {
        'estimator__n_estimators': np.linspace(50, 300, num = 10, dtype = int),
        'estimator__max_features': ['sqrt', 'log2', None],
        'estimator__criterion': ['gini', 'entropy', 'log_loss'],
        'estimator__min_samples_split': np.linspace(2, 20, num = 8, dtype = int),
        'encoder': encoders,
        'scaler': scalers
    },
    RandomForestClassifier(): {
        'estimator__n_estimators': np.linspace(50, 500, num = 10, dtype = int),
        'estimator__criterion': ['gini', 'entropy', 'log_loss'],
        'estimator__max_features': [sqrt', 'log2', None],
        'estimator__min_samples_split': np.linspace(2, 20, num = 10, dtype = int),
        'encoder': encoders,
        'scaler': scalers
    },
    AdaBoostClassifier(): {
        'estimator__estimator': [DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=3), DecisionTreeClassifier(max_depth=6), DecisionTreeClassifier(max_depth=10)],
        'estimator__n_estimators': np.linspace(50, 500, num = 10, dtype = int),
        'estimator__learning_rate': np.linspace(0.01, 1.0, num = 10, dtype = float),
        'encoder': encoders,
        'scaler': scalers
    },
    GradientBoostingClassifier(): {
        'estimator__n_estimators': np.linspace(50, 200, num = 4, dtype = int),
        'estimator__learning_rate': np.linspace(0.01, 1.0, num = 10, dtype = float),
        'estimator__max_depth': np.linspace(3, 10, num = 4, dtype = int),
        'encoder': encoders,
        'scaler': scalers
    },
    MLPClassifier(): {
        'estimator__hidden_layer_sizes': [(256,), (384,), (512,), (640,), (768,), (1024,)],
        'estimator__activation': ['identity', 'logistic', 'tanh', 'relu'],
        'estimator__solver': ['lbfgs', 'sgd', 'adam'],
        'estimator__max_iter': np.linspace(500, 1000, num = 15, dtype = int),
        'encoder': encoders,
        'scaler': scalers
    }
}

In [ ]:
def model_fit_eval(df, model, param_grid):

    evaldf = pd.DataFrame([])
    
    predictors = shuffle(data, random_state = 0).reset_index(drop=True)
    targets = predictors.pop('Collusive_competitor')

    steps = [
        ('encoder', ce.OneHotEncoder(cols = cat_features, handle_unknown = 'value', return_df = True)),
        ('scaler', preprocessing.StandardScaler()), 
        ('estimator', model)
    ]

    pipe = pipeline.Pipeline(steps)
    
    cv_outer = GroupKFold(n_splits = 5)
    cv_inner = GroupKFold(n_splits = 5)

    for i, (train_index, test_index) in enumerate(cv_outer.split(predictors, targets, predictors['Tender'])):
        x_train = predictors.loc[train_index]
        y_train = targets.loc[train_index]
        x_test = predictors.loc[test_index]
        y_test = targets.loc[test_index]
        
        groups = x_train['Tender']

        x_train.drop(columns = ['Tender'], inplace = True)
        x_test.drop(columns = ['Tender'], inplace = True)
        
        search = RandomizedSearchCV(
            pipe, 
            param_grid,
            cv = cv_inner,
            n_iter = 50,
            scoring=['accuracy','balanced_accuracy', 'precision', 'recall', 'f1'],
            refit = 'balanced_accuracy', 
            random_state = 0,
            n_jobs = -2
        )

        results = search.fit(x_train, y_train, groups = groups)

        df = pd.DataFrame([])

        df['Accuracy'] = pd.DataFrame(results.cv_results_).loc[results.best_index_][[
            'split0_test_accuracy', 'split1_test_accuracy', 'split2_test_accuracy', 'split3_test_accuracy', 'split4_test_accuracy'
        ]].reset_index(drop = True)*100
    
        df['Balanced Accuracy'] = pd.DataFrame(results.cv_results_).loc[results.best_index_][[
            'split0_test_balanced_accuracy', 'split1_test_balanced_accuracy', 'split2_test_balanced_accuracy',
            'split3_test_balanced_accuracy', 'split4_test_balanced_accuracy'
        ]].reset_index(drop = True)*100
    
        df['Precision'] = pd.DataFrame(results.cv_results_).loc[results.best_index_][[
            'split0_test_precision', 'split1_test_precision', 'split2_test_precision', 'split3_test_precision', 'split4_test_precision'
        ]].reset_index(drop = True)*100
    
        df['Recall'] = pd.DataFrame(results.cv_results_).loc[results.best_index_][[
            'split0_test_recall', 'split1_test_recall', 'split2_test_recall', 'split3_test_recall', 'split4_test_recall'
        ]].reset_index(drop = True)*100
    
        df['F1'] = pd.DataFrame(results.cv_results_).loc[results.best_index_][[
            'split0_test_f1', 'split1_test_f1', 'split2_test_f1', 'split3_test_f1', 'split4_test_f1'
        ]].reset_index(drop = True)*100
        
        df['CV'] = 'Inner'
        
        best_model = results.best_estimator_
        predictions = best_model.predict(x_test)

        acc = accuracy_score(y_test, predictions) * 100
        ba = balanced_accuracy_score(y_test, predictions) * 100
        pr = precision_score(y_test, predictions, pos_label=1, average='binary', zero_division=1) * 100
        rc = recall_score(y_test, predictions, pos_label=1, average='binary', zero_division=1) * 100
        f1 = f1_score(y_test, predictions, pos_label=1, average='binary', zero_division=1) * 100

        df = pd.concat([pd.DataFrame([[
            acc, 
            ba,
            pr,
            rc,
            f1,
            'Outer'
        ]], columns=df.columns), df], ignore_index=True)
        
        df['Outer Fold'] = i+1
        df['Model'] = model.__class__.__name__.replace('Classifier', '')
    
        for p in range(6):
            df.loc[p, 'Best Parameters'] = pd.DataFrame(results.cv_results_).loc[results.best_index_][['params']].reset_index(drop = True).values

        evaldf = pd.concat([evaldf, df], ignore_index=True)
    
    return evaldf

In [ ]:
start_time = time.time()
result = Parallel(n_jobs=-2)(delayed(model_fit_eval)(data, model, models[model]) for model in models)
total = time.time() - start_time

In [ ]:
total/60

In [ ]:
data_eval_all = pd.DataFrame([])

In [ ]:
for r in result:
    data_eval_all = pd.concat([data_eval_all, r], ignore_index=True)

In [ ]:
data_evall_all